<a href="https://colab.research.google.com/github/SuYenTing/Shopee-Code-League-2021/blob/main/shopee_address_elements_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shopee Code League - Address Elements Extraction
2021/03/18 蘇彥庭

* 程式碼改寫自: [Tensorflow學習課程-Neural machine translation with attention](https://www.tensorflow.org/tutorials/text/nmt_with_attention?fbclid=IwAR1JMaUDUpl0Q1T63YtoZIGsR7HqKpatLrd_oFzOwkcErJPK87zClprb6Ew)

In [ ]:
# 掛載Google雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 設定Google雲端資料夾(讀取數據及存放模型使用)
filePath = './drive/MyDrive/蝦皮競賽/'

In [ ]:
# 載入套件
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

import pandas as pd
from tqdm import tqdm 

In [ ]:
# 讀取訓練集數據
rawData = pd.read_csv(filePath + 'train.csv')
rawData

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru
...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [ ]:
# 觀察樣本
rawData.sample(20)

,id,raw_address,POI/street
139503,139503,lamda iii karawaci baru karawaci,/lamda iii
5177,5177,pondok pinang sus 11 rt 7 10 kebayoran lama,/sus
46873,46873,mojo klan lor 62 mojo rt 6 4 gubeng,/mojo klan lor
17002,17002,"niumich florist & party supplier, jl. siti arm...",depan smk wahana bakti/jl. siti armilah
46849,46849,jl. prof. dr. latume 19 gro permai off complex...,grogol permai office complex/jl. prof. dr. latume
97759,97759,"apartemen patria park, jl. di. panjaitan, note...",apartemen patria park/jl. di. panjaitan
45377,45377,"cuci mobil motor & kar, kap hary sardonoharjo",cuci mobil motor & karpet/kapten hary
268015,268015,"kup gn. tim., g v, putat jaya sawahan",kupang gn. tim./g v
248691,248691,sukomanunggal suko jaya i 64 3 sukomanunggal,/suko jaya i
56510,56510,ga8pro kem suka tani,ga8property/kem


In [ ]:
# 確認資料
checkData = rawData.iloc[[110083, 145144, 205578, 102960, 280843, 204764, 151962, 193572]]
checkData

In [ ]:
# 文本預處理
# 將raw_address的','改為' , '
# 此處保留逗號 因為對地址來說 應該是蠻重要的 可以區分不同類別(省份, 地標, 路, 街道, 號碼, etc)
rawData['raw_address'] = rawData['raw_address'].str.replace(',', ' , ')
# poi和street由於是預測目標 需完全相符才會得分 故不做預處理

# 處理POI/street斜線
rawData['POI/street'] = rawData['POI/street'].str.replace('/', ' <slash> ')

# 加入<start>和<end>字詞 讓模型知道何時開始預測與停止
rawData['raw_address'] = '<start> ' + rawData['raw_address'] + ' <end>'
rawData['POI/street'] = '<start> ' + rawData['POI/street'] + ' <end>'
rawData

In [ ]:
# tokenize函數
def tokenize(text):

  text_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  text_tokenizer.fit_on_texts(text)
  tensor = text_tokenizer.texts_to_sequences(text)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, text_tokenizer

In [ ]:
# 此處先只用raw_address預測POI/street欄位
# 對raw_address及POI/street做tokenize
target_tensor, targ_text = tokenize(rawData['POI/street'])
input_tensor, inp_text = tokenize(rawData['raw_address'])

In [ ]:
# token轉換文字函數
def convert(text, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, text.index_word[t]))

In [ ]:
# 驗證token轉換文字是否有問題
checkIdx = 300
print ("Input Text; index to word mapping")
convert(inp_text, input_tensor[checkIdx])
print ()
print ("Target Text; index to word mapping")
convert(targ_text, target_tensor[checkIdx])

In [ ]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
print('max_length_targ:', max_length_targ)
print('max_length_inp:', max_length_inp)

In [ ]:
# # Creating training and validation sets using an 80-20 split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=1234)

# # Show length
# print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

# 不做驗證集 直接訓練
input_tensor_train = input_tensor
target_tensor_train = target_tensor

In [ ]:
# Create a tf.data dataset
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_text.word_index)+1
vocab_tar_size = len(targ_text.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = filePath + 'model_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_text.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# # 模型預測
# def evaluate(inputs):

#   # input的token還原為文字
#   sentence = ''
#   for t in inputs:
#     if t!= 0:
#       sentence += inp_text.index_word[t] + ' '

#   attention_plot = np.zeros((max_length_targ, max_length_inp))

#   # output結果
#   result = ''

#   inputs = inputs[None, :]

#   hidden = [tf.zeros((1, units))]
#   enc_out, enc_hidden = encoder(inputs, hidden)

#   dec_hidden = enc_hidden
#   dec_input = tf.expand_dims([targ_text.word_index['<start>']], 0)

#   for t in range(max_length_targ):
#     predictions, dec_hidden, attention_weights = decoder(dec_input,
#                                                          dec_hidden,
#                                                          enc_out)

#     # storing the attention weights to plot later on
#     attention_weights = tf.reshape(attention_weights, (-1, ))
#     attention_plot[t] = attention_weights.numpy()

#     predicted_id = tf.argmax(predictions[0]).numpy()

#     result += targ_text.index_word[predicted_id] + ' '

#     if targ_text.index_word[predicted_id] == '<end>':
#       return result, sentence, attention_plot

#     # the predicted ID is fed back into the model
#     dec_input = tf.expand_dims([predicted_id], 0)

#   return result, sentence, attention_plot

In [ ]:
# # 範例
# result, sentence, attention_plot = evaluate(input_tensor_train[10])
# print('raw_address: ', sentence)
# print('predict POI/street: ', result)

In [ ]:
# # 模型預測
# def predict(inputs):

#   # output結果
#   result = np.array([], dtype='int32')

#   inputs = inputs[None, :]

#   hidden = [tf.zeros((1, units))]
#   enc_out, enc_hidden = encoder(inputs, hidden)

#   dec_hidden = enc_hidden
#   dec_input = tf.expand_dims([targ_text.word_index['<start>']], 0)

#   for t in range(max_length_targ):
#     predictions, dec_hidden, attention_weights = decoder(dec_input,
#                                                          dec_hidden,
#                                                          enc_out)

#     predicted_id = tf.argmax(predictions[0]).numpy()

#     result = np.append(result, predicted_id) 

#     if targ_text.index_word[predicted_id] == '<end>':
#       return result

#     # the predicted ID is fed back into the model
#     dec_input = tf.expand_dims([predicted_id], 0)

#   return result

In [ ]:
# # 評分 預測的token代碼需與實際的token代碼完全相符
# def evaluate(real, pred):

#   if len(real)-len(pred) < 0:
#     return 0
    
#   pred = np.pad(pred, (0, len(real)-len(pred)), 'constant', constant_values=0)

#   if np.sum(np.subtract(real, pred)) == 0:
#     return 1
#   else:
#     return 0

In [ ]:
# # 計算訓練集預測準確度
# trainScore = 0
# for i in range(len(target_tensor_train)):

#   score = evaluate(real=target_tensor_train[i][1:],  # 取1是為移除<start>標籤
#                    pred=predict(input_tensor_train[i]))
#   trainScore += score

In [ ]:
# # 計算訓練集準確度
# trainAcc = trainScore/len(target_tensor_train)
# print(f'訓練集預測poi準確度:{trainAcc}')

In [ ]:
# 處理預測集資料
# 讀取預測集數據
testData = pd.read_csv(filePath + 'test.csv')
testData

In [ ]:
# 文本預處理
# 將raw_address的','改為' , '
# 此處保留逗號 因為對地址來說 應該是蠻重要的 可以區分不同類別(省份, 地標, 路, 街道, 號碼, etc)
testData['raw_address'] = testData['raw_address'].str.replace(',', ' , ')
# poi和street由於是預測目標 需完全相符才會得分 故不做預處理

# 加入<start>和<end>字詞 讓模型知道何時開始預測與停止
testData['raw_address'] = '<start> ' + testData['raw_address'] + ' <end>'
testData

In [ ]:
# 模型預測預測集
def test_evaluate(sentence):

  inputs = [inp_text.word_index[i] for i in sentence.split(' ') if i in inp_text.word_index.keys()]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  # output結果
  result = ''

  # inputs = inputs[None, :]

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_text.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    if targ_text.index_word[predicted_id] == '<end>':
      result = result[:-1]  # 刪除最後的空白
      return result
    else:
      result += targ_text.index_word[predicted_id] + ' '

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result

In [ ]:
# 測試
test_evaluate(testData['raw_address'][55])

In [ ]:
# 進行預測
prdictResult = list()
for i in range(len(testData['raw_address'])):
  prdictResult.append(test_evaluate(testData['raw_address'][i]))
  if i % 50 == 0:
    print(f"目前進度: {i+1}/{len(testData['raw_address'])} ")

In [ ]:
# 整理預測結果
output = pd.DataFrame({'POI/street':prdictResult})
output['POI/street'] = output['POI/street'].str.replace(' <slash> ', '/')
output['POI/street'] = output['POI/street'].str.replace('<slash> ', '/')
output['POI/street'] = output['POI/street'].str.replace(' <slash>', '/')
output['POI/street'] = output['POI/street'].str.replace('<slash>', '/')
output['id'] = output.index
output = output[['id', 'POI/street']]
output

In [ ]:
# 儲存結果
output.to_csv(filePath + 'test_predict.csv', index=False)